In [15]:
import MySQLdb, sshtunnel, csv, bcrypt

In [28]:
BIND_PORT = 4360

In [29]:
tunnel = sshtunnel.open_tunnel(
    os.environ.get('SSH_HOST'),
    ssh_username=os.environ.get('SSH_USER'),
    ssh_password=os.environ.get('SSH_PASSWORD'),
    local_bind_address=('127.0.0.1', BIND_PORT),
    remote_bind_address=(os.environ.get('MYSQL_DB_HOST'), 3306)
)

In [27]:
with tunnel:
    db = MySQLdb.connect(
        user = os.environ.get('MYSQL_DB_USER'),
        password = os.environ.get('MYSQL_DB_PASSWORD'),
        database = os.environ.get('MYSQL_DB_DATABASE'),
        host = '127.0.0.1',
        port = BIND_PORT
    )
    cur = db.cursor()
    
    with open('../data/unicorn-keys.csv') as f:   
        for row in csv.reader(f):
            name = row[0]
            fullname = row[1]
            keyhash = bcrypt.hashpw(row[2].encode(), bcrypt.gensalt()).decode()
            cur = db.cursor()
            cur.execute(
                "insert into persons (person_name, person_fullname, secret_key) values (%s, %s, %s)",
                (name, fullname, keyhash)
            )

    db.commit()
    cur.close()
    db.close()